<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Deep-Learning-Projects/blob/main/Machine%20Translation%20-%20EN-IT%20-%20Tatoeba%20Dataset/Machine_Translation_Seq2Seq_Model_(Tatoeba_EN_IT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries & Setup Enviorment

In [1]:
# Install libs
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from datasets import load_dataset

In [3]:
# Setup Warning
import warnings
warnings.filterwarnings("ignore")

# Functions

In [14]:
# Loads tatoba from hugging face
def load_tatoeba_dataset():
    dataset = load_dataset("tatoeba", lang1="en", lang2="it", trust_remote_code=True)
    data = dataset['train']
    # استخراج زوج‌های جمله (انگلیسی و ایتالیایی)
    input_texts = [item['translation']['en'] for item in data]
    target_texts = [item['translation']['it'] for item in data]
    return input_texts, target_texts

# Prepares data
def prepare_data(input_texts, target_texts):
    # Adds <start> & <end> token
    target_texts = ["<start> " + text + " <end>" for text in target_texts]
    print("Input Samples: ", input_texts[:5])
    print("Output Samples: ", target_texts[:5])

    # Tokenization for inputs (EN)
    input_tokenizer = Tokenizer()
    input_tokenizer.fit_on_texts(input_texts)
    input_sequences = input_tokenizer.texts_to_sequences(input_texts)
    input_vocab_size = len(input_tokenizer.word_index) + 1
    max_input_len = max(len(seq) for seq in input_sequences)

    # Tokenization for outputs (FA)
    target_tokenizer = Tokenizer()
    target_tokenizer.fit_on_texts(target_texts)
    target_sequences = target_tokenizer.texts_to_sequences(target_texts)
    target_vocab_size = (len(target_tokenizer.word_index) + 1)
    max_target_len = max(len(seq) for seq in target_texts)

    # Padding
    encoder_input_data = pad_sequences(input_sequences, maxlen=max_input_len, padding='post')
    decoder_input_data = pad_sequences([seq[:-1] for seq in target_sequences], maxlen=max_target_len-1, padding='post')
    decoder_target_data = pad_sequences([seq[1:] for seq in target_sequences], maxlen=max_target_len-1, padding='post')

    decoder_target_data = np.expand_dims(decoder_target_data, -1)
    print("Shape encoder_input_data:", encoder_input_data.shape)
    print("Shape decoder_input_data:", decoder_input_data.shape)
    print("Shape decoder_target_data:", decoder_target_data.shape)

    return (encoder_input_data, decoder_input_data, decoder_target_data,
            input_vocab_size, target_vocab_size, max_input_len, max_target_len,
            input_tokenizer, target_tokenizer)

# Encoder
def build_encoder(input_vocab_size, max_input_len, embedding_dim, units):
    encoder_inputs = Input(shape=(max_input_len, ))
    encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(units, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]
    return encoder_inputs, encoder_states

# Decoder
def build_decoder(target_vocab_size, max_target_len, embedding_dim , units, encoder_states):
    decoder_inputs = Input(shape=(max_target_len - 1, ))
    decoder_embedding = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(target_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    return decoder_inputs, decoder_outputs

def build_model(encoder_inputs, decoder_inputs, decoder_outputs):
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


# Run Functions

In [15]:
# Loading Dataset
input_texts, target_texts = load_tatoeba_dataset()

# Sampling
input_texts = input_texts[:1000]
target_texts = target_texts[:1000]

# Preparing data
(encoder_input_data, decoder_input_data, decoder_target_data,
    input_vocab_size, target_vocab_size, max_input_len, max_target_len,
    input_tokenizer, target_tokenizer) = prepare_data(input_texts, target_texts)

# Params
EMBEDDING_DIM = 128
UNITS = 256

# Encoder
encoder_inputs, encoder_states = build_encoder(input_vocab_size, max_input_len, EMBEDDING_DIM, UNITS)
# Decoder
decoder_inputs, decoder_outputs = build_decoder(target_vocab_size, max_target_len, EMBEDDING_DIM, UNITS, encoder_states)

# Build model
model = build_model(encoder_inputs, decoder_inputs, decoder_outputs)
model.summary()

# Train the model
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,
    epochs=50,
    validation_split=0.2,
    verbose=1
)

# Save the model
model.save('seq2seq_model.h5')

Input Samples:  ["Let's try something.", 'I have to go to sleep.', 'I have to go to sleep.', "Today is June 18th and it is Muiriel's birthday!", "Today is June 18th and it is Muiriel's birthday!"]
Output Samples:  ['<start> Proviamo qualcosa! <end>', '<start> Devo andare a dormire. <end>', '<start> Io devo andare a dormire. <end>', '<start> Oggi è il 18 giugno ed è il compleanno di Muiriel! <end>', '<start> Oggi è il 18 di giugno ed è il compleanno di Muiriel! <end>']
Shape encoder_input_data: (1000, 23)
Shape decoder_input_data: (1000, 168)
Shape decoder_target_data: (1000, 168, 1)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 23)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 168)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 23, 128)   │    105,856 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 168, 128)  │    152,320 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    394,240 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 168,      │    394,240 │ embedding_1[0][0… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 168, 1190) │    305,830 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,352,486 (5.16 MB)

 Trainable params: 1,352,486 (5.16 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.8131 - loss: 3.8083 - val_accuracy: 0.9593 - val_loss: 0.2773
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.9554 - loss: 0.2887 - val_accuracy: 0.9600 - val_loss: 0.2587
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.9568 - loss: 0.2685 - val_accuracy: 0.9655 - val_loss: 0.2477
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9611 - loss: 0.2523 - val_accuracy: 0.9660 - val_loss: 0.2442
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9621 - loss: 0.2442 - val_accuracy: 0.9662 - val_loss: 0.2434
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9613 - loss: 0.2470 - val_accuracy: 0.9662 - val_loss: 0.2447
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9610 - loss: 0.2474 - val_accuracy: 0.9662 - val_loss: 0.2444
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.9608 - loss: 0.2441 - val_accuracy: 0.9662 - val_loss: